# Fluent Python: Chapter 7.
## Function Decorators and Closures

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [3]:
@deco
def target():
    print('running target()')

In [4]:
target()

running inner()


In [5]:
target

<function __main__.deco.<locals>.inner>